In [2]:
import sys, os, math, string, datetime as dt, time, re
import pandas as pd, numpy as np, matplotlib.pyplot as plt, sqlite3 as sql
import emosent, emoji, string

In [3]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk import sentiment
from nltk import RegexpParser

In [4]:
tree = os.walk("../data/raw")

In [5]:
for folders, subfolders, files in tree:
    print(folders)
    print('--')
    print(subfolders)
    print('----')
    print(files)
    print("\n")

../data/raw
--
['BullTrades', 'Learn-n', 'Legacy Trading', 'LSV', 'Misc', 'OmegaTrades', 'Pennybois', 'Phantom Trades', 'Templar Trading', 'WayTrade']
----
['.gitkeep', 'intrinsicValueCalculator.xlsx', 'text2.db', 'text3.db', 'word_test.csv', '~$intrinsicValueCalculator.xlsx']


../data/raw\BullTrades
--
[]
----
['options-chat.db', 'stocks-chat.db']


../data/raw\Learn-n
--
[]
----
['trade-chat.db']


../data/raw\Legacy Trading
--
[]
----
['general.db', 'profits.db', 'text.db', 'trading-discussion.db']


../data/raw\LSV
--
[]
----
['options.db', 'trading.db']


../data/raw\Misc
--
[]
----
['stock-chat.db', 'text.db', 'text2.db', 'text3.db', 'text4.db', 'text5.db', 'text6.db', 'trade-ideas.db', 'wetlqd-ideas.db']


../data/raw\OmegaTrades
--
[]
----
['general.db', 'potentialdupe.db', 'sentiment.db', 'tech.db', 'text.db', 'winning.db']


../data/raw\Pennybois
--
[]
----
['stock-check.db']


../data/raw\Phantom Trades
--
[]
----
['lt-investing.db', 'trading-floor.db']


../data/raw\Templa

In [6]:
def getalldata(folder):
    master_df = pd.DataFrame(columns=["id", "name", "content", "timestamp"])
    tree = os.walk(folder)
    for folders, subfolder, files in tree:
        if folders == "../data/raw":
            next
        else:
            
            for file in files: # get database file
                connection_str = folders.replace("\\", "/") +"/" + file #setup connection string
                connect = sql.connect(connection_str)
                mycur = connect.cursor()
                mycur.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name") #get available tables
                available_tables = mycur.fetchall()
                server_name = folders.replace("../data/raw\\", "") #server name to append to df
                channel_name = file.replace(".db", "") # channel name to append to df
                for i, tbl in enumerate(available_tables):
                    df = pd.read_sql(f"SELECT * FROM {tbl[0]}", con=connect)
                    df["channel"] = channel_name
                    df["server"] = server_name
                    master_df = pd.concat([master_df, df], axis=0, ignore_index=True)
                connect.close()
                print(f"processed {channel_name}. Beep Bop Boop.....")
    return master_df

In [7]:
total_df = getalldata("../data/raw")

processed options-chat. Beep Bop Boop.....
processed stocks-chat. Beep Bop Boop.....
processed trade-chat. Beep Bop Boop.....
processed general. Beep Bop Boop.....
processed profits. Beep Bop Boop.....
processed text. Beep Bop Boop.....
processed trading-discussion. Beep Bop Boop.....
processed options. Beep Bop Boop.....
processed trading. Beep Bop Boop.....
processed stock-chat. Beep Bop Boop.....
processed text. Beep Bop Boop.....
processed text2. Beep Bop Boop.....
processed text3. Beep Bop Boop.....
processed text4. Beep Bop Boop.....
processed text5. Beep Bop Boop.....
processed text6. Beep Bop Boop.....
processed trade-ideas. Beep Bop Boop.....
processed wetlqd-ideas. Beep Bop Boop.....
processed general. Beep Bop Boop.....
processed potentialdupe. Beep Bop Boop.....
processed sentiment. Beep Bop Boop.....
processed tech. Beep Bop Boop.....
processed text. Beep Bop Boop.....
processed winning. Beep Bop Boop.....
processed stock-check. Beep Bop Boop.....
processed lt-investing. B

In [8]:
total_df.server.unique()

array(['BullTrades', 'Learn-n', 'Legacy Trading', 'LSV', 'Misc',
       'OmegaTrades', 'Pennybois', 'Phantom Trades', 'Templar Trading',
       'WayTrade'], dtype=object)

In [9]:
total_df.channel.unique()

array(['options-chat', 'stocks-chat', 'trade-chat', 'general', 'profits',
       'text', 'trading-discussion', 'options', 'trading', 'stock-chat',
       'text2', 'text3', 'text4', 'text5', 'text6', 'trade-ideas',
       'wetlqd-ideas', 'potentialdupe', 'sentiment', 'tech', 'winning',
       'stock-check', 'lt-investing', 'trading-floor', 'day-trading',
       'main-trading', 'option-trading', 'general-trading'], dtype=object)

In [10]:
total_df.server = total_df.server.apply(lambda x: x if x != "Misc" else "misc")

In [11]:
total_df = total_df.sort_values(["server", "channel"])

In [12]:
total_df.drop_duplicates(subset=["id", "content"])[lambda x: (x.server == "misc")].channel.value_counts()

text6    4905
text       38
Name: channel, dtype: int64

## Dealing with Duplicate Dbs in /Misc

In [13]:
# stock-chat.db = Legacy Trading/profits.db = text3.db
total_df[lambda x: x.channel == "stock-chat"].drop_duplicates(subset=["id", "content"]).sort_values("timestamp")

,id,name,content,timestamp,channel,server
59572,187336630749757440,Cyborg#8226,Damn nice,2020-10-14T02:30:06.383000+00:00,stock-chat,misc
59567,187336630749757440,Cyborg#8226,What’d you pay for it,2020-10-14T02:30:29.588000+00:00,stock-chat,misc
59563,290014935499538435,toan112#9456,FB looks good,2020-10-14T02:30:32.627000+00:00,stock-chat,misc
59559,290014935499538435,toan112#9456,2.55 as it said,2020-10-14T02:30:42.804000+00:00,stock-chat,misc
59555,719919449863356430,bernie#3240,510 for both contracts,2020-10-14T02:30:46.699000+00:00,stock-chat,misc
...,...,...,...,...,...,...
57272,700394878462328832,Jaime#0289,AMD is wedging and this supply of confluent wi...,2020-12-08T15:18:52.685000+00:00,stock-chat,misc
57268,700394878462328832,Jaime#0289,/NQ at a fresh 4h demands hopefully we can see...,2020-12-08T15:21:21.489000+00:00,stock-chat,misc
57265,700394878462328832,Jaime#0289,/ES and /NQ are flying. No idea why equities a...,2020-12-08T16:03:05.514000+00:00,stock-chat,misc
57266,479056760255021056,.Austin#4801,I’ll be posting some juicy charting and more p...,2020-12-08T16:37:44.868000+00:00,stock-chat,misc


In [14]:
(total_df[lambda x: (x.channel == "profits")&(x.server == "Legacy Trading")]).drop_duplicates(subset=["id", "content"]).sort_values("timestamp")

,id,name,content,timestamp,channel,server
41234,187336630749757440,Cyborg#8226,Damn nice,2020-10-14T02:30:06.383000+00:00,profits,Legacy Trading
41229,187336630749757440,Cyborg#8226,What’d you pay for it,2020-10-14T02:30:29.588000+00:00,profits,Legacy Trading
41225,290014935499538435,toan112#9456,FB looks good,2020-10-14T02:30:32.627000+00:00,profits,Legacy Trading
41221,290014935499538435,toan112#9456,2.55 as it said,2020-10-14T02:30:42.804000+00:00,profits,Legacy Trading
41217,719919449863356430,bernie#3240,510 for both contracts,2020-10-14T02:30:46.699000+00:00,profits,Legacy Trading
...,...,...,...,...,...,...
38934,700394878462328832,Jaime#0289,AMD is wedging and this supply of confluent wi...,2020-12-08T15:18:52.685000+00:00,profits,Legacy Trading
38930,700394878462328832,Jaime#0289,/NQ at a fresh 4h demands hopefully we can see...,2020-12-08T15:21:21.489000+00:00,profits,Legacy Trading
38927,700394878462328832,Jaime#0289,/ES and /NQ are flying. No idea why equities a...,2020-12-08T16:03:05.514000+00:00,profits,Legacy Trading
38928,479056760255021056,.Austin#4801,I’ll be posting some juicy charting and more p...,2020-12-08T16:37:44.868000+00:00,profits,Legacy Trading


In [15]:
# verdict: text3.db = stock-chat = Legacy Trading/profits
total_df[lambda x: x.channel == "text3"].drop_duplicates(subset=["id", "content"]).sort_values("timestamp")

,id,name,content,timestamp,channel,server
66444,187336630749757440,Cyborg#8226,Damn nice,2020-10-14T02:30:06.383000+00:00,text3,misc
66439,187336630749757440,Cyborg#8226,What’d you pay for it,2020-10-14T02:30:29.588000+00:00,text3,misc
66435,290014935499538435,toan112#9456,FB looks good,2020-10-14T02:30:32.627000+00:00,text3,misc
66431,290014935499538435,toan112#9456,2.55 as it said,2020-10-14T02:30:42.804000+00:00,text3,misc
66427,719919449863356430,bernie#3240,510 for both contracts,2020-10-14T02:30:46.699000+00:00,text3,misc
...,...,...,...,...,...,...
64144,700394878462328832,Jaime#0289,AMD is wedging and this supply of confluent wi...,2020-12-08T15:18:52.685000+00:00,text3,misc
64140,700394878462328832,Jaime#0289,/NQ at a fresh 4h demands hopefully we can see...,2020-12-08T15:21:21.489000+00:00,text3,misc
64137,700394878462328832,Jaime#0289,/ES and /NQ are flying. No idea why equities a...,2020-12-08T16:03:05.514000+00:00,text3,misc
64138,479056760255021056,.Austin#4801,I’ll be posting some juicy charting and more p...,2020-12-08T16:37:44.868000+00:00,text3,misc


<h3 style="color:red">verdict: delete text3.db and stock-chat.db from /Misc/</h3>

In [16]:
# OmegaTrades/potentialdupe = misc/trade-ideas.db = misc/text5.db 
total_df[lambda x: x.channel == "potentialdupe"].drop_duplicates(subset=["id", "content"]).sort_values("timestamp")

,id,name,content,timestamp,channel,server
159074,673279340326420549,wes10#3165,https://twitter.com/realdonaldtrump/status/131...,2020-10-06T18:54:07.769000+00:00,potentialdupe,OmegaTrades
159068,673279340326420549,wes10#3165,https://www.bloomberg.com/amp/news/articles/20...,2020-11-05T01:54:47.610000+00:00,potentialdupe,OmegaTrades
159069,548320800202293258,SargeMaximus#4491,So why are stocks mooning with this election bs?,2020-11-06T03:02:40.575000+00:00,potentialdupe,OmegaTrades
159070,375968444803186698,kwatson20#2892,> So why are stocks mooning with this election...,2020-11-06T04:03:00.627000+00:00,potentialdupe,OmegaTrades
159063,625730781683122218,TraderMatt#9564,Biden’s lead is good for big tech,2020-11-06T04:03:46.984000+00:00,potentialdupe,OmegaTrades
...,...,...,...,...,...,...
158558,451481789110681631,Zachary1022#1017,holy shit read that yall,2020-12-04T21:46:05.806000+00:00,potentialdupe,OmegaTrades
158554,548320800202293258,SargeMaximus#4491,"Crazy, why do you think whoever it was sold?",2020-12-05T03:22:51.893000+00:00,potentialdupe,OmegaTrades
158551,451481789110681631,Zachary1022#1017,<@!548320800202293258> bc they think the marke...,2020-12-05T03:23:12.691000+00:00,potentialdupe,OmegaTrades
158552,451481789110681631,Zachary1022#1017,"nah but idk, they always sell before we know,t...",2020-12-05T03:24:23.736000+00:00,potentialdupe,OmegaTrades


In [17]:
# the trade ideas databate contains more unique comments
total_df[lambda x: x.channel == "trade-ideas"].drop_duplicates(subset=["id", "content"]).sort_values("timestamp")

,id,name,content,timestamp,channel,server
110841,673279340326420549,wes10#3165,https://twitter.com/realdonaldtrump/status/131...,2020-10-06T18:54:07.769000+00:00,trade-ideas,misc
111887,673279340326420549,wes10#3165,GME halted,2020-10-08T18:07:49.653000+00:00,trade-ideas,misc
111878,673279340326420549,wes10#3165,https://www.google.com/amp/s/www.bloomberg.com...,2020-10-10T20:10:08.122000+00:00,trade-ideas,misc
111873,673279340326420549,wes10#3165,Waymo is an autonomous driving tech company un...,2020-10-10T20:11:13.856000+00:00,trade-ideas,misc
111868,673279340326420549,wes10#3165,https://t.co/u4ZbtW9JX5?amp=1 \n\n\nThe Compan...,2020-10-14T19:48:17.976000+00:00,trade-ideas,misc
...,...,...,...,...,...,...
110325,451481789110681631,Zachary1022#1017,holy shit read that yall,2020-12-04T21:46:05.806000+00:00,trade-ideas,misc
110321,548320800202293258,SargeMaximus#4491,"Crazy, why do you think whoever it was sold?",2020-12-05T03:22:51.893000+00:00,trade-ideas,misc
110318,451481789110681631,Zachary1022#1017,<@!548320800202293258> bc they think the marke...,2020-12-05T03:23:12.691000+00:00,trade-ideas,misc
110319,451481789110681631,Zachary1022#1017,"nah but idk, they always sell before we know,t...",2020-12-05T03:24:23.736000+00:00,trade-ideas,misc


In [18]:
# trade-ideas = text5.db 
total_df[lambda x: (x.channel == "text5") & (x.server == "misc")].drop_duplicates(subset=["id", "content"]).sort_values("timestamp")

,id,name,content,timestamp,channel,server
71008,673279340326420549,wes10#3165,https://twitter.com/realdonaldtrump/status/131...,2020-10-06T18:54:07.769000+00:00,text5,misc
72054,673279340326420549,wes10#3165,GME halted,2020-10-08T18:07:49.653000+00:00,text5,misc
72045,673279340326420549,wes10#3165,https://www.google.com/amp/s/www.bloomberg.com...,2020-10-10T20:10:08.122000+00:00,text5,misc
72040,673279340326420549,wes10#3165,Waymo is an autonomous driving tech company un...,2020-10-10T20:11:13.856000+00:00,text5,misc
72035,673279340326420549,wes10#3165,https://t.co/u4ZbtW9JX5?amp=1 \n\n\nThe Compan...,2020-10-14T19:48:17.976000+00:00,text5,misc
...,...,...,...,...,...,...
70492,451481789110681631,Zachary1022#1017,holy shit read that yall,2020-12-04T21:46:05.806000+00:00,text5,misc
70488,548320800202293258,SargeMaximus#4491,"Crazy, why do you think whoever it was sold?",2020-12-05T03:22:51.893000+00:00,text5,misc
70485,451481789110681631,Zachary1022#1017,<@!548320800202293258> bc they think the marke...,2020-12-05T03:23:12.691000+00:00,text5,misc
70486,451481789110681631,Zachary1022#1017,"nah but idk, they always sell before we know,t...",2020-12-05T03:24:23.736000+00:00,text5,misc


In [19]:
# OmegaTrades/potentialdupe = partial duplicate of misc/trade-ideas.db = misc/text5.db that are partial duplicates of text.db
total_df[lambda x: (x.channel == "text") & (x.server == "misc")].drop_duplicates(subset=["id", "content"]).sort_values("timestamp")

,id,name,content,timestamp,channel,server
60604,673279340326420549,wes10#3165,https://twitter.com/realdonaldtrump/status/131...,2020-10-06T18:54:07.769000+00:00,text,misc
60598,673279340326420549,wes10#3165,https://www.bloomberg.com/amp/news/articles/20...,2020-11-05T01:54:47.610000+00:00,text,misc
60599,548320800202293258,SargeMaximus#4491,So why are stocks mooning with this election bs?,2020-11-06T03:02:40.575000+00:00,text,misc
60600,375968444803186698,kwatson20#2892,> So why are stocks mooning with this election...,2020-11-06T04:03:00.627000+00:00,text,misc
60593,625730781683122218,TraderMatt#9564,Biden’s lead is good for big tech,2020-11-06T04:03:46.984000+00:00,text,misc
...,...,...,...,...,...,...
62112,374316215797284877,! - ƓŎ ƦᗴŞᚡĄᙡŊ - !#4955,FSLY,2020-12-08T15:35:58.628000+00:00,text,misc
62105,374316215797284877,! - ƓŎ ƦᗴŞᚡĄᙡŊ - !#4955,HYLN,2020-12-08T15:40:38.188000+00:00,text,misc
62102,374316215797284877,! - ƓŎ ƦᗴŞᚡĄᙡŊ - !#4955,Cheap leap that will guarantee a profit. IVR r...,2020-12-08T16:10:49.635000+00:00,text,misc
62100,374316215797284877,! - ƓŎ ƦᗴŞᚡĄᙡŊ - !#4955,Very bullish. Broke out and held it's place ab...,2020-12-08T16:24:27.489000+00:00,text,misc


<h3 style="color:red">verdict: delete OmegaTrades/potentialdupe.db, Misc/text5.db and ./trade-ideas.db + rename: text.db to trade-ideas and mv to /OmegaTrades</h3>

In [20]:
# Omega Trades/winning = misc/text2.db
total_df[lambda x: (x.channel == "text2") & (x.server == "misc")].drop_duplicates(subset=["id", "content"])

,id,name,content,timestamp,channel,server
63613,271708801609105408,dylskrrt#0864,another good call,2020-12-08T15:18:53.907000+00:00,text2,misc
63614,673279340326420549,wes10#3165,PLTR this morning and a DKNG daytrade,2020-12-08T16:35:06.936000+00:00,text2,misc
63617,673279340326420549,wes10#3165,Quick trade on DDOG,2020-12-04T16:32:40.061000+00:00,text2,misc
63620,281194229676376065,A_Lucky_Pucker#8227,200 scalping DOCU\nLots of other trades that u...,2020-12-04T16:11:53.140000+00:00,text2,misc
63623,673279340326420549,wes10#3165,Snap lottos 🙂,2020-12-03T16:58:01.890000+00:00,text2,misc
...,...,...,...,...,...,...
64107,281194229676376065,A_Lucky_Pucker#8227,250 on AMD quick scalp play this morning just ...,2020-10-26T13:39:52.625000+00:00,text2,misc
64117,281194229676376065,A_Lucky_Pucker#8227,<@!673279340326420549> Thank you sir ^^,2020-10-21T22:38:42.152000+00:00,text2,misc
64122,281194229676376065,A_Lucky_Pucker#8227,$60.00 profit. \nBut bad risk to reward ratio\...,2020-10-21T15:23:17.722000+00:00,text2,misc
64124,281194229676376065,A_Lucky_Pucker#8227,Entered too early then exited to early,2020-10-21T14:58:06.478000+00:00,text2,misc


In [21]:
# Winning has a few more comments
(total_df[lambda x: (x.channel == "winning")&(x.server == "OmegaTrades")]).drop_duplicates(subset=["id", "content"])

,id,name,content,timestamp,channel,server
167637,271708801609105408,dylskrrt#0864,another good call,2020-12-08T15:18:53.907000+00:00,winning,OmegaTrades
167638,673279340326420549,wes10#3165,PLTR this morning and a DKNG daytrade,2020-12-08T16:35:06.936000+00:00,winning,OmegaTrades
167641,673279340326420549,wes10#3165,Quick trade on DDOG,2020-12-04T16:32:40.061000+00:00,winning,OmegaTrades
167644,281194229676376065,A_Lucky_Pucker#8227,200 scalping DOCU\nLots of other trades that u...,2020-12-04T16:11:53.140000+00:00,winning,OmegaTrades
167647,673279340326420549,wes10#3165,Snap lottos 🙂,2020-12-03T16:58:01.890000+00:00,winning,OmegaTrades
...,...,...,...,...,...,...
168181,673279340326420549,wes10#3165,Let’s go hippo base hits not home runs,2020-10-14T14:49:11.936000+00:00,winning,OmegaTrades
168183,531986968091951105,Hippo#3098,It’s not much but I’m starting to get the ball...,2020-10-14T14:48:51.657000+00:00,winning,OmegaTrades
168189,531986968091951105,Hippo#3098,Apple 116-123,2020-10-13T15:48:46.812000+00:00,winning,OmegaTrades
168192,281194229676376065,A_Lucky_Pucker#8227,512.90 x 80\n513.19 x 79,2020-10-13T15:47:09.237000+00:00,winning,OmegaTrades


<h3 style="color:red">verdict: delete Misc/text2.db</h3>

In [22]:
# text.db = text4.db
total_df[lambda x: (x.channel == "text") & (x.server == "misc")].drop_duplicates(subset=["id", "content"])

,id,name,content,timestamp,channel,server
59577,451481789110681631,Zachary1022#1017,<@!548320800202293258> bc they think the marke...,2020-12-05T03:23:12.691000+00:00,text,misc
59578,451481789110681631,Zachary1022#1017,"nah but idk, they always sell before we know,t...",2020-12-05T03:24:23.736000+00:00,text,misc
59579,548320800202293258,SargeMaximus#4491,Ah true,2020-12-05T03:24:26.764000+00:00,text,misc
59580,548320800202293258,SargeMaximus#4491,"Crazy, why do you think whoever it was sold?",2020-12-05T03:22:51.893000+00:00,text,misc
59584,451481789110681631,Zachary1022#1017,holy shit read that yall,2020-12-04T21:46:05.806000+00:00,text,misc
...,...,...,...,...,...,...
63588,374316215797284877,! - ƓŎ ƦᗴŞᚡĄᙡŊ - !#4955,AAPL: Wait for a break to either the upside or...,2020-11-21T22:29:21.835000+00:00,text,misc
63593,374316215797284877,! - ƓŎ ƦᗴŞᚡĄᙡŊ - !#4955,"MSFT: Trading Sideways, wait for break",2020-11-21T22:29:10.607000+00:00,text,misc
63598,159985870458322944,MEE6#4876,"GG <@!374316215797284877>, you just advanced t...",2020-11-21T22:29:00.484000+00:00,text,misc
63603,374316215797284877,! - ƓŎ ƦᗴŞᚡĄᙡŊ - !#4955,"Costco: Ascending triangle, watch for a break ...",2020-11-21T22:29:00.368000+00:00,text,misc


In [23]:
total_df[lambda x: (x.channel == "text4") & (x.server == "misc")].drop_duplicates(subset=["id", "content"])

,id,name,content,timestamp,channel,server
66449,451481789110681631,Zachary1022#1017,<@!548320800202293258> bc they think the marke...,2020-12-05T03:23:12.691000+00:00,text4,misc
66450,451481789110681631,Zachary1022#1017,"nah but idk, they always sell before we know,t...",2020-12-05T03:24:23.736000+00:00,text4,misc
66451,548320800202293258,SargeMaximus#4491,Ah true,2020-12-05T03:24:26.764000+00:00,text4,misc
66452,548320800202293258,SargeMaximus#4491,"Crazy, why do you think whoever it was sold?",2020-12-05T03:22:51.893000+00:00,text4,misc
66456,451481789110681631,Zachary1022#1017,holy shit read that yall,2020-12-04T21:46:05.806000+00:00,text4,misc
...,...,...,...,...,...,...
70460,374316215797284877,! - ƓŎ ƦᗴŞᚡĄᙡŊ - !#4955,AAPL: Wait for a break to either the upside or...,2020-11-21T22:29:21.835000+00:00,text4,misc
70465,374316215797284877,! - ƓŎ ƦᗴŞᚡĄᙡŊ - !#4955,"MSFT: Trading Sideways, wait for break",2020-11-21T22:29:10.607000+00:00,text4,misc
70470,159985870458322944,MEE6#4876,"GG <@!374316215797284877>, you just advanced t...",2020-11-21T22:29:00.484000+00:00,text4,misc
70475,374316215797284877,! - ƓŎ ƦᗴŞᚡĄᙡŊ - !#4955,"Costco: Ascending triangle, watch for a break ...",2020-11-21T22:29:00.368000+00:00,text4,misc


<h3 style="color:red">remove Misc/text4.db</h3>

In [24]:
# wetlqd-ideas = text6.db
total_df[lambda x: (x.channel == "text6") & (x.server == "misc")].drop_duplicates(subset=["id", "content"])

,id,name,content,timestamp,channel,server
72058,488359813160501251,Merente#5844,Thanks <@464449226521313292> ! I opted for th...,2020-11-30T20:48:43.035000+00:00,text6,misc
72059,689681229989150742,jackaplan#7289,Barely followed the market at all today (didn’...,2020-11-30T20:58:32.872000+00:00,text6,misc
72060,612337363326074936,Trademate#2515,<@131952052069728258> has reached level **13**!,2020-11-30T19:55:20.913000+00:00,text6,misc
72063,131952052069728258,Teecee#3022,<@!464449226521313292> thanks boss,2020-11-30T19:55:20.625000+00:00,text6,misc
72067,464449226521313292,TradingJEDI#2647,<@!399198468159045633> BRO AWSOME SO GLAD YOU ...,2020-11-30T19:44:26.516000+00:00,text6,misc
...,...,...,...,...,...,...
110279,445583268792893449,itsyoboyvic#7198,Heroin,2019-08-02T01:14:44.367000+00:00,text6,misc
110284,445583268792893449,itsyoboyvic#7198,Big crack,2019-08-02T01:14:39.682000+00:00,text6,misc
110289,445583268792893449,itsyoboyvic#7198,Crack,2019-08-02T01:14:35.798000+00:00,text6,misc
110294,579205396166737921,beenpippinsincebeenpippin#4039,@everyone today’s total. This doesn’t calculat...,2019-08-02T01:13:47.923000+00:00,text6,misc


In [25]:
total_df[lambda x: (x.channel == "wetlqd-ideas") & (x.server == "misc")].drop_duplicates(subset=["id", "content"])

,id,name,content,timestamp,channel,server
111891,488359813160501251,Merente#5844,Thanks <@464449226521313292> ! I opted for th...,2020-11-30T20:48:43.035000+00:00,wetlqd-ideas,misc
111892,689681229989150742,jackaplan#7289,Barely followed the market at all today (didn’...,2020-11-30T20:58:32.872000+00:00,wetlqd-ideas,misc
111893,612337363326074936,Trademate#2515,<@131952052069728258> has reached level **13**!,2020-11-30T19:55:20.913000+00:00,wetlqd-ideas,misc
111896,131952052069728258,Teecee#3022,<@!464449226521313292> thanks boss,2020-11-30T19:55:20.625000+00:00,wetlqd-ideas,misc
111900,464449226521313292,TradingJEDI#2647,<@!399198468159045633> BRO AWSOME SO GLAD YOU ...,2020-11-30T19:44:26.516000+00:00,wetlqd-ideas,misc
...,...,...,...,...,...,...
150112,445583268792893449,itsyoboyvic#7198,Heroin,2019-08-02T01:14:44.367000+00:00,wetlqd-ideas,misc
150117,445583268792893449,itsyoboyvic#7198,Big crack,2019-08-02T01:14:39.682000+00:00,wetlqd-ideas,misc
150122,445583268792893449,itsyoboyvic#7198,Crack,2019-08-02T01:14:35.798000+00:00,wetlqd-ideas,misc
150127,579205396166737921,beenpippinsincebeenpippin#4039,@everyone today’s total. This doesn’t calculat...,2019-08-02T01:13:47.923000+00:00,wetlqd-ideas,misc


<h3 style="color:red">remove Misc/text6.db</h3>

<h3>/raw/data Cleanup</h3>
<ul>
<li>rm: text6, text5, text4, text3, text2, stock-chat, trade-ideas, OmegaTrades/potentialDup</li>
<li>rename: misc/text.db -> Omega Trades/trade-ideas.db</li>
</ul>

In [34]:
total_df_no_misc = getalldata("../data/raw")

processed options-chat. Beep Bop Boop.....
processed stocks-chat. Beep Bop Boop.....
processed trade-chat. Beep Bop Boop.....
processed general. Beep Bop Boop.....
processed profits. Beep Bop Boop.....
processed text. Beep Bop Boop.....
processed trading-discussion. Beep Bop Boop.....
processed options. Beep Bop Boop.....
processed trading. Beep Bop Boop.....
processed wetlqd-ideas. Beep Bop Boop.....
processed general. Beep Bop Boop.....
processed sentiment. Beep Bop Boop.....
processed tech. Beep Bop Boop.....
processed text. Beep Bop Boop.....
processed trade-ideas. Beep Bop Boop.....
processed winning. Beep Bop Boop.....
processed stock-check. Beep Bop Boop.....
processed lt-investing. Beep Bop Boop.....
processed trading-floor. Beep Bop Boop.....
processed day-trading. Beep Bop Boop.....
processed main-trading. Beep Bop Boop.....
processed option-trading. Beep Bop Boop.....
processed general-trading. Beep Bop Boop.....


In [35]:
total_df_no_misc = total_df_no_misc.drop_duplicates(["id", "content"])

In [36]:
total_df_no_misc.channel.unique()

array(['options-chat', 'stocks-chat', 'trade-chat', 'general', 'profits',
       'text', 'trading-discussion', 'options', 'trading', 'wetlqd-ideas',
       'sentiment', 'tech', 'trade-ideas', 'winning', 'stock-check',
       'lt-investing', 'trading-floor', 'day-trading', 'main-trading',
       'option-trading', 'general-trading'], dtype=object)

In [74]:
total_df_no_misc = total_df_no_misc.sort_values("timestamp").reset_index(drop=True)

In [75]:
total_df_no_misc.to_sql("comments", con="sqlite:///../data/interim/discord/discord.db", index_label="pk")

In [80]:
total_df_no_misc.shape

(31934, 6)

<h3>Comment Cleaning</h3>
    <ul>
        <li>Starting Point: 31,934 Unique Comments</li>
        <li>In order to get these comments ready for NLP analysis, I must:</li>
        <ul>
        <li>Remove scripted comments (those automatically genreated by bots)</li>
        <li>Identify Single Words Making Up Comments That Are Key to This Analysis:</li>
            <ul>
            <li>Emojis</li>
            <li>Specific Companies and Their Tickers</li>
            <li>Stock Trading Specific Slang</li>
            <li>User Mentions</li>
            </ul>
        </ul>
    </ul>

In [90]:
def catchBot(c):
    botCom = re.compile("\shas reached level \*\*.\d+\*\*")
    bot2Com = re.compile("\<.*\>, you just advanced to")
    botreg = botCom.search(c)
    botreg1 = bot2Com.search(c.strip())
    if (botreg != None) or (botreg1 != None):
        return 1
    else:
        return 0

#### Find Bots

In [91]:
total_df_no_misc = total_df_no_misc.assign(isBot = lambda x: x.content.apply(catchBot))

In [246]:
bots = total_df_no_misc[lambda x: x.isBot == 1].id.unique()

In [257]:
bots = np.concatenate([bots, total_df_no_misc[lambda x: x.name.str.contains("Bot#")].id.unique()], axis=0)

In [258]:
total_df_no_misc.loc[(total_df_no_misc.id.isin(bots)), "isBot"] = 1

#### Find Mentions

In [344]:
def catchMentions(comment):
    userIdCatcherT = r"(<@(!)?\d+>|@everyone)"
    mention = re.findall(userIdCatcherT, comment)
    if mention != []:
        return ",".join([re.sub(r"[<>!@]", "", x[i][0]) for i, x in enumerate([mention])])
    else:
        return ""

In [345]:
total_df = total_df_no_misc.assign(mentions = lambda x: x.content.apply(catchMentions))

#### Find Emojis

In [346]:
def catchEmojis(comment):
    emoji_list = emoji.emoji_lis(comment)
    return ",".join([e["emoji"] for e in emoji_list])

In [347]:
total_df = total_df.assign(emojis = lambda x: x.content.apply(catchEmojis))

In [348]:
total_df[lambda x: x.emojis != ""].head()

,id,name,content,timestamp,channel,server,isBot,mentions,emojis
12,408706621942792192,zeuse4848#9335,😃,2019-08-02T17:55:31.789000+00:00,wetlqd-ideas,Misc,0,,😃
41,570291052494323722,Moocah#1918,Griffin McDonald’s is all doodoo 😂😂 <@57920539...,2019-08-08T17:02:22.143000+00:00,wetlqd-ideas,Misc,0,579205396166737921,"😂,😂"
80,456226577798135808,Deleted User#0000,🔥🔥😳😳,2019-08-16T15:19:59.020000+00:00,wetlqd-ideas,Misc,0,,"🔥,🔥,😳,😳"
87,587888385197342732,ricky#8577,😂,2019-08-16T16:37:53.126000+00:00,wetlqd-ideas,Misc,0,,😂
97,611594621222387738,BreadGame#4233,Good morning everyone 😃,2019-08-19T13:34:26.860000+00:00,wetlqd-ideas,Misc,0,,😃


In [382]:
total_df.to_sql("comments", con="sqlite:///../data/interim/discord/discord.db", index_label="pk", if_exists="replace")

#### Find Companies
* find companies using name, ticker or associated words
    * This project goal is to match and process comment sentiment and match it with the particular companies mentioned.
    * Companies are signified by a variety of words including: their traded ticker abbreviation, their full company name and by the names of members of their C-Suite.
    * In order to accurately and systematically obtain these mentioned companies, I will have to create a simple list of companies based on single words (ticker and name).
    * Once I have an understanding of the most popular companies being talked about in the groups, I can create a more refined parsing function to identify words that most likely trace back to the company.
    * I want to maintain the structure of the comments throughout the analysis (maintain initial structure), so that my NLP analysis can grade sentiment based on the original comment's grammer and word usage

In [349]:
# Discords link to stock starts with a $, but can only pull out 338 comments out of 30,000+. 
total_df[lambda x: x.content.str.contains(r"\$[A-Za-z]")]

,id,name,content,timestamp,channel,server,isBot,mentions,emojis
246,367506726582484996,JK#0643,"Yea, literally all thanks to you guys. Just wa...",2019-08-29T21:06:45.315000+00:00,wetlqd-ideas,Misc,0,,
610,445583268792893449,itsyoboyvic#7198,$EXAS,2019-10-02T19:12:03.224000+00:00,wetlqd-ideas,Misc,0,,
1372,449633258116874250,dakotared#3395,quick scalp on $SCWX with Dell buying out the ...,2019-12-23T18:48:28.342000+00:00,wetlqd-ideas,Misc,0,,
1921,674648888279302204,GOOFYSPY#8332,$IRM got small position. Will hold it overnight.,2020-02-25T20:58:08.310000+00:00,wetlqd-ideas,Misc,0,,
1930,674648888279302204,GOOFYSPY#8332,"$Dis, interesting 🤔",2020-02-26T17:04:51.441000+00:00,wetlqd-ideas,Misc,0,,🤔
...,...,...,...,...,...,...,...,...,...
31855,396160546140979212,jmorri#6551,$GRIL again today <@!101806058359496704>,2020-12-08T20:13:38.815000+00:00,main-trading,Templar Trading,0,101806058359496704,
31856,101806058359496704,Ryan (itsGoggles)#8580,$CANG up too <:peepoyay:735222523683405984>,2020-12-08T20:13:38.984000+00:00,main-trading,Templar Trading,0,,
31858,396160546140979212,jmorri#6551,$AESE for anyone that was playing it just went...,2020-12-08T20:13:53.474000+00:00,main-trading,Templar Trading,0,,
31889,101806058359496704,Ryan (itsGoggles)#8580,$RUN i had on a watchlist like last week but t...,2020-12-08T20:20:47.630000+00:00,option-trading,Templar Trading,0,,


In [89]:
company_regex = re.compile(r"[A-Z]{2,4}")

In [380]:
# identifying companies based on tickers. Better representative. 338 -> 6,438 
total_df[lambda x:(x.content.str.contains(company_regex))]

,id,name,content,timestamp,channel,server,isBot,mentions,emojis
23,445583268792893449,itsyoboyvic#7198,I took stamps .... and IAC both not looking go...,2019-08-08T03:31:33.793000+00:00,wetlqd-ideas,Misc,0,,
26,194305752939102208,D.Ninan#3776,Sold AMD aug23rd call for a fat profit,2019-08-08T13:33:00.968000+00:00,wetlqd-ideas,Misc,0,,
47,194305752939102208,D.Ninan#3776,MSFT profits im still not selling it tho just ...,2019-08-13T14:21:58.192000+00:00,wetlqd-ideas,Misc,0,,
54,344157708523667457,dylan.patel#4692,$139.50 and ATH,2019-08-13T19:01:16.234000+00:00,wetlqd-ideas,Misc,0,,
56,194305752939102208,D.Ninan#3776,$139.50 and ATH? Wym,2019-08-13T19:05:27.213000+00:00,wetlqd-ideas,Misc,0,,
...,...,...,...,...,...,...,...,...,...
31910,428921853403463713,Baraxton#0199,@everyone EDIT is popping off,2020-12-08T20:25:14.542000+00:00,trading,LSV,0,everyone,
31916,785007528152399913,nickb34#9557,$OXY broke the resistance line on Brad's chart,2020-12-08T20:26:51.453000+00:00,trading,LSV,0,,
31919,594593113104842795,VelcroKarma#1526,if PIXY runs from being included in the Benzin...,2020-12-08T20:27:59.342000+00:00,trade-chat,Learn-n,0,,
31930,184420431242067977,Dg#8853,<@428921853403463713> have you done a video in...,2020-12-08T20:39:17.246000+00:00,trading,LSV,0,428921853403463713,


In [4]:
# next to do:
    # gather list of companies on the four American exchanges: NYSE and NASDAQ and their tickers (available via NASDAQ's website)
    # have dictionary parser that translates company name mentions to its assocaited ticker.
    # with the name and tickers, a more thorough pulling of the appropiate companies can take place. The pulling should take place on the str.lower value of the content column.
    # company mentions will be stored just as emojis and mentions are stored for now. (as comma-separated values column)

In [4]:
with sql.connect("../data/interim/discord/discord.db") as con:
    total_df = pd.read_sql("SELECT id, name, content, timestamp, channel, server, isBot, mentions, emojis FROM comments", con=con)

In [5]:
tickers_df = pd.DataFrame()
for fold, _, files in os.walk("../data/external/stock_data/"):
    for f in files:
        if f.startswith("nasdaq_"):
            _temp_df = pd.read_csv(fold+"/"+f)
            _temp_df = _temp_df.assign(exchange = f.split("_")[-1].replace(".csv", "").upper())
            tickers_df = pd.concat([tickers_df, _temp_df], ignore_index=True, axis=0)

In [6]:
tickers_df

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry,exchange
0,AAU,Almaden Minerals Ltd. Common Shares,$0.318,-0.0009,-0.282%,4.363641e+07,Canada,2015.0,196663,NaN,NaN,AMEX
1,ACU,Acme United Corporation. Common Stock,$31.47,-1.6500,-4.982%,1.120234e+08,United States,1988.0,5792,Consumer Durables,Home Furnishings,AMEX
2,ACY,AeroCentury Corp. Common Stock,$35.00,0.0100,0.029%,1.545884e+08,United States,NaN,36527,Technology,Diversified Commercial Services,AMEX
3,ADRA,Adara Acquisition Corp. Class A Common Stock,$9.80,0.0191,0.195%,1.408750e+08,United States,2021.0,401,NaN,NaN,AMEX
4,AE,Adams Resources & Energy Inc. Common Stock,$28.14,-0.4100,-1.436%,1.225497e+08,United States,NaN,1218,Energy,Oilfield Services/Equipment,AMEX
...,...,...,...,...,...,...,...,...,...,...,...,...
8183,ZTS,Zoetis Inc. Class A Common Stock,$227.85,2.1500,0.953%,1.078017e+11,United States,2013.0,641943,Health Care,Biotechnology: Pharmaceutical Preparations,NYSE
8184,ZUO,Zuora Inc. Class A Common Stock,$20.32,0.3500,1.753%,2.531872e+09,NaN,2018.0,190253,Technology,EDP Services,NYSE
8185,ZVIA,Zevia PBC Class A Common Stock,$8.50,0.6900,8.835%,2.928799e+08,NaN,2021.0,84708,Consumer Non-Durables,Specialty Foods,NYSE
8186,ZWS,Zurn Water Solutions Corporation Common Stock,$36.87,0.0500,0.136%,4.474157e+09,United States,2012.0,105372,Public Utilities,Water Supply,NYSE


In [7]:
tickers_df.dropna(subset=["Market Cap"]).sort_values(["Market Cap"], ascending=False)

2528 | INFO | Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2528 | INFO | NumExpr defaulting to 8 threads.


,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry,exchange
284,AAPL,Apple Inc. Common Stock,$174.225,3.0450,1.779%,3.020598e+12,United States,1980.0,67838373,Technology,Computer Manufacturing,NASDAQ
3171,MSFT,Microsoft Corporation Common Stock,$331.5383,-3.3817,-1.01%,2.489183e+12,United States,1986.0,11093825,Technology,Computer Software: Prepackaged Software,NASDAQ
2162,GOOG,Alphabet Inc. Class C Capital Stock,$2964.03,3.3000,0.111%,1.967416e+12,United States,2004.0,418187,Technology,Internet and Information Services,NASDAQ
2163,GOOGL,Alphabet Inc. Class A Common Stock,$2946.56,1.1700,0.04%,1.955820e+12,United States,NaN,448053,Technology,Internet and Information Services,NASDAQ
531,AMZN,Amazon.com Inc. Common Stock,$3519.10,-4.1900,-0.119%,1.784703e+12,United States,1997.0,1220708,Consumer Services,Catalog/Specialty Distribution,NASDAQ
...,...,...,...,...,...,...,...,...,...,...,...,...
4942,XFINU,ExcelFin Acquisition Corp Unit,$10.05,0.0000,0.00%,0.000000e+00,United States,2021.0,1,NaN,NaN,NASDAQ
2316,HIIIW,Hudson Executive Investment Corp. III Warrant,$0.77,-0.0300,-3.75%,0.000000e+00,United States,2021.0,1275,NaN,NaN,NASDAQ
4952,XOMAO,XOMA Corporation Depositary Shares Rep Series ...,$26.25,-0.1500,-0.568%,0.000000e+00,United States,NaN,5488,Health Care,Biotechnology: Pharmaceutical Preparations,NASDAQ
4953,XOMAP,XOMA Corporation 8.625% Series A Cumulative Pe...,$26.35,-0.0500,-0.189%,0.000000e+00,United States,NaN,140,Health Care,Biotechnology: Pharmaceutical Preparations,NASDAQ


In [8]:
tickers_df["Market Cap"].value_counts()

0.000000e+00    1245
4.209000e+08       4
2.808875e+08       4
3.363750e+08       3
2.823250e+08       3
                ... 
2.670478e+08       1
2.729275e+09       1
5.984373e+08       1
3.534605e+09       1
1.907935e+08       1
Name: Market Cap, Length: 6435, dtype: int64

In [9]:
classifiable_companies = tickers_df.dropna(subset=["Market Cap"])[lambda x: x["Market Cap"] > 0].sort_values(["Market Cap"], ascending=False).reset_index(drop=True)

In [10]:
classifiable_companies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6470 entries, 0 to 6469
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Symbol      6470 non-null   object 
 1   Name        6470 non-null   object 
 2   Last Sale   6470 non-null   object 
 3   Net Change  6470 non-null   float64
 4   % Change    6469 non-null   object 
 5   Market Cap  6470 non-null   float64
 6   Country     5802 non-null   object 
 7   IPO Year    3879 non-null   float64
 8   Volume      6470 non-null   int64  
 9   Sector      5560 non-null   object 
 10  Industry    5575 non-null   object 
 11  exchange    6470 non-null   object 
dtypes: float64(3), int64(1), object(8)
memory usage: 606.7+ KB


In [11]:
def findCommonWords(stock_array):
    common_words = {}
    for stock in stock_array:
        for word in stock.lower().split():
            if word in common_words.keys():
                common_words[word] += 1
            else:
                common_words[word] = 1
        
    return common_words


In [12]:
company_words = findCommonWords(classifiable_companies.Name.values)

In [13]:
company_word = dict(sorted(company_words.items(), key=lambda x: x[1], reverse=True))

In [14]:
company_word

{'common': 5288,
 'stock': 4881,
 'inc.': 3333,
 'shares': 1227,
 'class': 1125,
 'a': 1101,
 'corporation': 791,
 'ordinary': 633,
 'corp.': 552,
 'holdings': 416,
 'acquisition': 395,
 'american': 371,
 'fund': 346,
 'group': 315,
 'depositary': 310,
 'income': 240,
 'ltd.': 236,
 'limited': 236,
 'therapeutics': 210,
 'company': 199,
 'of': 197,
 'trust': 185,
 'inc': 160,
 'beneficial': 152,
 'financial': 148,
 'interest': 147,
 '&': 145,
 'representing': 139,
 'technologies': 137,
 'energy': 129,
 'pharmaceuticals': 128,
 'capital': 127,
 'international': 127,
 'bancorp': 126,
 'share': 112,
 'global': 112,
 'plc': 105,
 'municipal': 96,
 'holding': 88,
 'ii': 84,
 'technology': 80,
 'partners': 80,
 '(the)': 78,
 'incorporated': 77,
 'and': 70,
 'corp': 70,
 's.a.': 69,
 'first': 69,
 'health': 67,
 'new': 66,
 'industries': 63,
 'each': 62,
 'investment': 61,
 'one': 59,
 'blackrock': 58,
 'services': 57,
 'co.': 56,
 'systems': 56,
 'units': 56,
 'nuveen': 55,
 'resources': 52,

In [15]:
legal_phrases_re = re.compile(r"common stock|class a|ltd.|limited|common shares|ordinary shares|class b|class c|class d|holdings|inc\.|capital stock|corporation|corp\.|company|incorporated|depositary shares|\(the\)|group holding|\(de\)")

In [16]:
classifiable_companies = classifiable_companies.assign(single_name = lambda x: x.Name.apply(lambda comp: re.sub(legal_phrases_re, "", comp.lower())))

In [17]:
tickers_and_names = classifiable_companies.loc[:, ["Symbol", "Name", "single_name"]]

In [18]:
tickers_and_names.head(20)

,Symbol,Name,single_name
0,AAPL,Apple Inc. Common Stock,apple
1,MSFT,Microsoft Corporation Common Stock,microsoft
2,GOOG,Alphabet Inc. Class C Capital Stock,alphabet
3,GOOGL,Alphabet Inc. Class A Common Stock,alphabet
4,AMZN,Amazon.com Inc. Common Stock,amazon.com
5,TSLA,Tesla Inc. Common Stock,tesla
6,FB,Meta Platforms Inc. Class A Common Stock,meta platforms
7,NVDA,NVIDIA Corporation Common Stock,nvidia
8,TSM,Taiwan Semiconductor Manufacturing Company Ltd.,taiwan semiconductor manufacturing
9,JPM,JP Morgan Chase & Co. Common Stock,jp morgan chase & co.


In [19]:
tickers_and_names.sample(20, random_state=11)

,Symbol,Name,single_name
3113,SGML,Sigma Lithium Corporation Common Shares,sigma lithium
2621,NBHC,National Bank Holdings Corporation Common Stock,national bank
1295,DCP,DCP Midstream LP Common Units,dcp midstream lp common units
2649,PTNR,Partner Communications Company Ltd. American D...,partner communications american
2521,NE,Noble Corporation plc Ordinary Shares,noble plc
1490,TLRY,Tilray Inc. Class 2 Common Stock,tilray class 2
53,MS,Morgan Stanley Common Stock,morgan stanley
5954,OPGN,OpGen Inc. Common Stock,opgen
1787,COOP,Mr. Cooper Group Inc. Common Stock,mr. cooper group
6166,ESP,Espey Mfg. & Electronics Corp. Common Stock,espey mfg. & electronics


In [20]:
stop_words = set(stopwords.words("english"))

In [21]:
# understand basic linguistic structure of comments. Words used. To see how I should approach picking out companies and analyzing sentiment
from nltk.tokenize import word_tokenize, sent_tokenize

In [22]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [23]:
re.sub(r"[\’\'\'\"\"\“\”!\?@#$%&\(\)\*,-.\\\{\}+~\/:;<>\[\]^`|=_]", "", "’”!#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~")

''

In [24]:
def wordCounts(comment_array):
    comment_dictionary = {} 
    for comment in comment_array:
        punc_regex = r"[\’\'\'\"\"\“\”!\?@#$%&\(\)\*,-.\\\{\}+~\/:;<>\[\]^`|=_]"
        comment = comment.lower()
        comment = re.sub(punc_regex, "", comment)
        comment = re.sub(r"[0-9]", "", comment)
        emoji_list = emoji.emoji_lis(comment)
        emoji_list = [e["emoji"] for e in emoji_list]
        words = word_tokenize(comment)
        for w in words:
            if (w in comment_dictionary.keys()) & (w not in stop_words) & (w not in emoji_list):
                comment_dictionary[w] += 1
            elif (w not in stop_words) & (w not in emoji_list):
                comment_dictionary[w] = 1
    comments_words = dict(sorted(comment_dictionary.items(), key=lambda x: x[1], reverse=True))
    
    return pd.DataFrame.from_dict(comments_words, orient="index", columns=["count"]).reset_index().rename(columns={"index": "word"})

In [25]:
wordy_df = wordCounts(total_df.content.values)

In [26]:
wordy_df = wordy_df.assign(no_vow = lambda x: x.word.apply(lambda x: 1 if re.findall(r"[aeiou]+", x) != [] else 0))

In [27]:
total_df[lambda x: x.content.str.contains("nflx")]

,id,name,content,timestamp,channel,server,isBot,mentions,emojis
483,310277096683601920,BassgodTV#2261,bad entry into nflx,2019-09-19T20:01:42.511000+00:00,wetlqd-ideas,Misc,0,,
777,579205396166737921,beenpippinsincebeenpippin#4039,Some more legs of that nflx earnings play comi...,2019-10-18T16:02:04.230000+00:00,wetlqd-ideas,Misc,0,,
2675,579205396166737921,beenpippinsincebeenpippin#4039,Just playing around small with the iv shifts ...,2020-04-21T15:00:34.317000+00:00,wetlqd-ideas,Misc,0,,
6958,443880858529693716,OnionKnight#2425,Trimming from nflx gain today,2020-09-15T13:59:14.966000+00:00,wetlqd-ideas,Misc,0,,
8641,480583337661038602,DCruz#3069,> Has anyone taken a look at nflx\n<@713103211...,2020-09-27T03:22:20.051000+00:00,options-chat,BullTrades,0,713103211141136602,🙂
15937,748645854482792570,tdub2322#9571,!ta nflx,2020-10-24T06:00:48.017000+00:00,stock-check,Pennybois,0,,
15938,748645854482792570,tdub2322#9571,!cd nflx,2020-10-24T06:01:15.377000+00:00,stock-check,Pennybois,0,,
22725,636343146246307846,Chabias_Trades#3203,I’m in nflx and dhr for large cap swings,2020-11-15T04:53:08.813000+00:00,options-chat,BullTrades,0,,
27708,734512402124636251,rawitg#6093,!cd nflx,2020-11-29T01:40:49.301000+00:00,stock-check,Pennybois,0,,
30546,499289176945197057,kdbar#4223,I prefer dis if you're looking at nflx. They w...,2020-12-06T01:55:34.569000+00:00,option-trading,Templar Trading,0,,


In [28]:
# !abbv represent commands said in the channel in order to get a response from specific bot 
total_df[lambda x: x.content.str.contains(r"!ta")].iloc[0, 2]

'<@744455713732165652> if you go to the stockbot room and type !ta stockname it will tell you the support.... That would be a good entry point if you can get in around there'

In [29]:
total_df[lambda x: x.content.str.contains(r"!ta")].iloc[0, :]

id                                          719195862869475360
name                                             Sarizzle#3273
content      <@744455713732165652> if you go to the stockbo...
timestamp                     2020-09-23T02:10:33.705000+00:00
channel                                             trade-chat
server                                                 Learn-n
isBot                                                        0
mentions                                    744455713732165652
emojis                                                        
Name: 7995, dtype: object

In [30]:
total_df.loc[7992:8005, "content"]

7992                   <@!719195862869475360> yes 1 month
7993    <@744455713732165652> Awesome I am going on fo...
7994    <@!719195862869475360> I’m learning about that...
7995    <@744455713732165652> if you go to the stockbo...
7996                                              !ta aks
7997                                          For example
7998                                                ☝️👁️👍
7999                                             !ta aapl
8000               <@!719195862869475360> here’s my play!
8001                   Do you have both a put and a call?
8002                                    I am in AMD also!
8003                      <@!719195862869475360> yes I do
8004                           I just noticed iron condor
8005                   Lol it’s ok I do that all the time
Name: content, dtype: object

In [31]:
total_df[lambda x: x.content.str.contains(r"![a-z]+")]

,id,name,content,timestamp,channel,server,isBot,mentions,emojis
3350,716994215862599744,sunny1293#1607,!alert <@!579205396166737921>,2020-06-17T16:59:41.393000+00:00,wetlqd-ideas,Misc,0,579205396166737921,
5200,200451257566101505,nzo#0913,!rank,2020-08-28T01:41:04.718000+00:00,general,OmegaTrades,0,,
5220,507295065308397588,shumps#9758,!rank,2020-08-28T01:56:52.242000+00:00,general,OmegaTrades,0,,
5235,476692246801022976,t0ny32#1039,!futures,2020-08-28T03:05:54.882000+00:00,general-trading,WayTrade,0,,
5312,366373605853888515,baby#4110,!rank,2020-08-28T15:40:14.679000+00:00,general,OmegaTrades,0,,
...,...,...,...,...,...,...,...,...,...
31673,731738204662923284,hippoQQ#2886,!ta fsr,2020-12-08T18:50:04.472000+00:00,stock-check,Pennybois,0,,
31691,715388996456415293,yellowserverguy#9736,!ta IBIO,2020-12-08T19:23:56.585000+00:00,stock-check,Pennybois,0,,
31694,764933310375526400,Cashman36#5176,!ta snss,2020-12-08T19:31:04.429000+00:00,stock-check,Pennybois,0,,
31698,726547154348408884,Rakesh#5604,!ta prog,2020-12-08T19:39:00.636000+00:00,stock-check,Pennybois,0,,


In [32]:
total_df.loc[5190:5205, :]

,id,name,content,timestamp,channel,server,isBot,mentions,emojis
5190,717204128673169532,BigBucktShaq#3361,I got that 9/11 call,2020-08-28T00:45:05.384000+00:00,general-trading,WayTrade,0,,
5191,743310228816134145,Ghosty 👻#0670,It's a shame oracle didn't have the bread to c...,2020-08-28T00:45:44.935000+00:00,general-trading,WayTrade,0,,
5192,743310228816134145,Ghosty 👻#0670,💀fat L,2020-08-28T00:45:53.339000+00:00,general-trading,WayTrade,0,,💀
5193,743310228816134145,Ghosty 👻#0670,Lmao imagine being a tiktok exec 💀,2020-08-28T00:46:38.291000+00:00,general-trading,WayTrade,0,,💀
5194,743310228816134145,Ghosty 👻#0670,Got every major player at your knees 🤑,2020-08-28T00:47:00.224000+00:00,general-trading,WayTrade,0,,🤑
5195,387329318021365774,endowdly#0373,> <@!387329318021365774> I thought that said 8...,2020-08-28T00:47:02.155000+00:00,wetlqd-ideas,Misc,0,387329318021365774,
5196,387329318021365774,endowdly#0373,less small (still small) action today,2020-08-28T00:49:17.227000+00:00,wetlqd-ideas,Misc,0,,
5197,622169977314869282,Riq#2278,Nah I didn’t <@717204128673169532>,2020-08-28T00:50:53.591000+00:00,general-trading,WayTrade,0,717204128673169532,
5198,250629554115313664,stamford™#1812,<@!712352964542595083> what app is that?,2020-08-28T01:13:12.071000+00:00,general-trading,WayTrade,0,712352964542595083,
5199,712352964542595083,KyaleTheChef#1672,Investing.com <@250629554115313664>,2020-08-28T01:15:57.596000+00:00,general-trading,WayTrade,0,250629554115313664,


In [33]:
wordy_df[lambda x: x.no_vow <1].head(30)

,word,count,no_vow
28,k,514,0
69,spy,313,0
205,c,134,0
213,cd,130,0
218,rn,128,0
231,try,124,0
290,pltr,98,0
312,btc,90,0
316,dkng,89,0
328,gg,86,0


In [34]:
total_df[lambda x: x.content.str.contains(r"\s*jks\s*")]

,id,name,content,timestamp,channel,server,isBot,mentions,emojis
14176,636343146246307846,Chabias_Trades#3203,plz die jks,2020-10-19T03:00:23.742000+00:00,stocks-chat,BullTrades,0,,
14184,546379668463288330,clashofclans2000#9874,"se is more linear, jks is just parabolic",2020-10-19T03:07:42.264000+00:00,stocks-chat,BullTrades,0,,
14220,758030851366846546,purplefloridian(andrew)#8481,Someone asked me if they should jump in jks,2020-10-19T03:30:40.311000+00:00,stocks-chat,BullTrades,0,,
14438,546379668463288330,clashofclans2000#9874,<@!636343146246307846> sell puts on jks,2020-10-19T22:56:18.191000+00:00,stocks-chat,BullTrades,0,636343146246307846,
14782,636343146246307846,Chabias_Trades#3203,i have to kill jks,2020-10-21T01:35:27.635000+00:00,stocks-chat,BullTrades,0,,
15055,387830168665849858,johnjacobs#4668,!ta jks,2020-10-21T17:15:09.184000+00:00,stock-check,Pennybois,0,,
15395,546379668463288330,clashofclans2000#9874,jks is insane,2020-10-22T21:49:51.976000+00:00,stocks-chat,BullTrades,0,,
15467,712352964542595083,KyaleTheChef#1672,im interested in jks now,2020-10-23T02:28:54.377000+00:00,general-trading,WayTrade,0,,
15615,546379668463288330,clashofclans2000#9874,impossible to predict jks,2020-10-23T04:03:58.887000+00:00,stocks-chat,BullTrades,0,,
15681,218856794896859139,Duki#1711,I also have a side of 50k in the brokerage I d...,2020-10-23T18:20:26.611000+00:00,stocks-chat,BullTrades,0,,


In [35]:
total_df[lambda x: x.content.str.contains("amzn")].sample(1, random_state=13).content

24583    20k is worth it in 2 years amzn will be soarin...
Name: content, dtype: object

In [57]:
from nltk.help import upenn_tagset

In [60]:
import nltk

In [62]:
nltk.download("tagsets")

[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\Kyle\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping help\tagsets.zip.


True

In [70]:
sample_tokens = word_tokenize(total_df[lambda x: x.content.str.contains("amzn")].sample(1, random_state=6).content.values[0])
sample_pos_tags = pos_tag(sample_tokens)

In [71]:
pattern = 'NP: {<DT>?<JJ>*<NN>}'

In [72]:
comment_parse = RegexpParser(pattern)
tree = comment_parse.parse(sample_pos_tags)

In [73]:
print(tree)

(S
  Was/NNP
  (NP b****in/NN)
  about/IN
  amzn/JJ
  (NP all/DT day/NN)
  and/CC
  then/RB
  sold/VBN
  at/IN
  breakeven/JJ
  </NNS
  :/:
  KEKW:756336180555022458/NNP
  (NP >/NN))


In [69]:
tree.draw()

#### Find Slang 